# ÇOKLU DOĞRUSAL REGRESYON

In [3]:
import pandas as pd

df = pd.read_csv("Advertising.csv")
df = df.iloc[:, 1:len(df)]
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [12]:
X = df.drop("sales", axis=1)
y = df[["sales"]]  # iki adet iç içe köşeli parantez

In [13]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


In [14]:
y.head()

,sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


### Statsmodels ile model kurmak

In [19]:
import statsmodels.api as sm

In [23]:
linearModel = sm.OLS(y, X)

In [24]:
model = linearModel.fit()

In [26]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              3566.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                   2.43e-171
Time:                        20:56:35   Log-Likelihood:                         -423.54
No. Observations:                 200   AIC:                                      853.1
Df Residuals:                     197   BIC:                                      863.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0538      0.001     40.507      0.000       0.051       0.056
radio          0.2222      0.009     23.595      0.000       0.204       0.241
newspaper      0.0168      0.007      2.517      0.013       0.004       0.030
==============================================================================
Omnibus:                        5.982   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.039
Skew:                          -0.232   Prob(JB):                       0.0296
Kurtosis:                       3.794   Cond. No.                         12.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Scikit Learn ile Model Kurmak

In [27]:
from sklearn.linear_model import LinearRegression

In [29]:
linearModel = LinearRegression()

In [30]:
model = linearModel.fit(X, y)

In [31]:
model.intercept_

array([2.93888937])

In [32]:
model.coef_

array([[ 0.04576465,  0.18853002, -0.00103749]])

### Tahmin Modeli Kurma

In [33]:
yeni_veri = [[243], [121], [52]]

In [36]:
yeni_veri = pd.DataFrame(yeni_veri).T

In [37]:
yeni_veri

,0,1,2
0,243,121,52


In [38]:
model.predict(yeni_veri)

array([[36.81788062]])

In [41]:
2.93888937 + 0.04576465*243 + 0.18853002*121 - 0.00103749*52

36.81788226

In [42]:
from sklearn.metrics import mean_squared_error

In [45]:
MSE = mean_squared_error(y, model.predict(X))
MSE

2.784126314510936

In [47]:
import numpy as np
RMSE = np.sqrt(MSE)
RMSE

1.6685701407225697

### Model Tuning (Model Doğrulama)

In [79]:
# sınama seti
from sklearn.model_selection import train_test_split

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1)

In [105]:
X_train.head()

,TV,radio,newspaper
108,13.1,0.4,25.6
107,90.4,0.3,23.2
189,18.7,12.1,23.4
14,204.1,32.9,46.0
56,7.3,28.1,41.4


In [106]:
X_test.head()

,TV,radio,newspaper
58,210.8,49.6,37.7
40,202.5,22.3,31.6
34,95.7,1.4,7.4
102,280.2,10.1,21.4
184,253.8,21.3,30.0


In [107]:
y_train.head()

,sales
108,5.3
107,8.7
189,6.7
14,19.0
56,5.5


In [108]:
y_test.head()

,sales
58,23.8
40,16.6
34,9.5
102,14.8
184,17.6


In [109]:
linearModel = LinearRegression().fit(X_train, y_train)

In [110]:
# Eğitim Hatası
RMSE = np.sqrt(mean_squared_error(y_train, linearModel.predict(X_train)))
RMSE

1.736902590147092

In [111]:
# Test Hatası
RMSE = np.sqrt(mean_squared_error(y_test, linearModel.predict(X_test)))
RMSE

1.4113417558581582

### K-KATLI CROSSVALIDATION

In [112]:
from sklearn.model_selection import cross_val_score

In [113]:
cross_val_score(linearModel, X_train, y_train, cv= 15, scoring="neg_mean_squared_error")  # "neg_mean_squared_error" == hata kareler ortalaması 

array([-1.98825437, -1.46498602, -5.06105166, -3.64764435, -1.54958062,
       -3.69671641, -2.38411435, -8.19516007, -3.79633822, -1.91558739,
       -4.14665407, -2.61441983, -1.16826336, -5.23395728, -1.75456443])

In [118]:
# cv --> MSE --> train
(-cross_val_score(linearModel, X_train, y_train, cv= 15, scoring="neg_mean_squared_error")).mean()

3.2411528286758133

In [120]:
# cv --> RMSE --> train
np.sqrt((-cross_val_score(linearModel, X_train, y_train, cv= 15, scoring="neg_mean_squared_error")).mean())

1.8003202017074111

In [117]:
# cv--> MSE --> test
(-cross_val_score(linearModel, X_test, y_test, cv= 15, scoring="neg_mean_squared_error")).mean()

1.6306505601471641

In [121]:
# cv --> RMSE --> test
np.sqrt((-cross_val_score(linearModel, X_test, y_test, cv= 15, scoring="neg_mean_squared_error")).mean())

1.276969287080611